In [ ]:
import cv2
import numpy as np
import h5py
import argparse
from pathlib import Path
import roboticstoolbox as rtb
import json
import open3d as o3d
from multiprocessing.shared_memory import SharedMemory
import numpy as np
from scipy.spatial.transform import Rotation as R
import pyquaternion as pyq  
from scipy.optimize import minimize
import json
import os


def wait_for_data(width, height, control_data:np.ndarray, data_shm:SharedMemory):
    while True:
        try:
            if control_data[0] <= 0.5:
                buffer = np.ndarray((height, width, 3), dtype=np.uint8, buffer=data_shm.buf)
                image = np.array(buffer)
                return image
        except Exception as e:
            print(control_data)
            raise e

def send_by_shm(control_data:np.ndarray, cam_pose, fx, fy, width, height):
    cam_trans, cam_rot = cam_pose
    control_data[1:4] = cam_trans
    control_data[4:8] = cam_rot
    control_data[8] = fx
    control_data[9] = fy
    control_data[10] = width
    control_data[11] = height
    control_data[0] = 1.0
    
def get_rendered_images(cam_pose, height, width, fx, fy):
    control_shm = SharedMemory(name='control_psm_08d5dd701')
    data_shm = SharedMemory(name="data_psm_08d5dd701")
    control_data = np.ndarray((12,), dtype=np.float64, buffer=control_shm.buf)
    position = cam_pose[0]
    rotation = cam_pose[1]
    send_by_shm(control_data, cam_pose, fx, fy, width, height)
    image = wait_for_data(width, height, control_data, data_shm)
    return image

def get_to_marker_pose(image, camera_params, tag_size, detector, min_num=4):
    '''
    get the pose: camera -> marker
    '''
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    results = detector.detect(gray, estimate_tag_pose=True, camera_params=camera_params, tag_size=tag_size)
    if len(results) < min_num:
        return None
    res = None
    for result in results:
        if result.tag_id == 22222:
            res = result
            break
    if res is None:
        return None
    pose = np.eye(4)
    pose[:3, :3] = res.pose_R
    pose[:3, 3] = res.pose_t[:, 0]
    pose = np.linalg.inv(pose)
    hope2now = np.eye(4)
    hope2now[1, 1] = -1
    hope2now[2, 2] = -1
    now2hope = np.linalg.inv(hope2now)
    tmp_trans = np.eye(4)
    tmp_trans[2, 2] = -1
    pose = now2hope @ pose 
    return pose

def create_camera_model(size=0.1):
    mesh_camera = o3d.geometry.TriangleMesh.create_coordinate_frame(size=size, origin=[0, 0, 0])
    # mesh_camera.paint_uniform_color([0.9, 0.1, 0.1])
    return mesh_camera

def show_pose(camera_pose, size=0.1):
    camera_pose = np.array(camera_pose)
    camera_pose[:3, :3] = camera_pose[:3, :3] / np.abs((np.linalg.det(camera_pose[:3, :3]))) ** (1/3)
    tmp_tans = np.eye(4)
    tmp_tans[2, 2] = -1
    # camera_pose =  camera_pose @ tmp_tans
    camera_model = create_camera_model(size)
    camera_model.transform(camera_pose)
    return camera_model

def filter_and_compute_mean_std(matrices, outlier_percent=0.1):
    """
    Filter out a given percentage of outlier matrices and compute the mean and standard deviation.

    Parameters:
    matrices (numpy.ndarray): A numpy array of shape (n, m, m) representing n m x m matrices.
    outlier_percent (float): The percentage of outliers to remove (0 < outlier_percent < 0.5), default is to remove the top and bottom 10%.

    Returns:
    mean_matrix (numpy.ndarray): The mean matrix after filtering.
    std_matrix (numpy.ndarray): The standard deviation matrix after filtering.
    """
    # Step 1: Compute the norm of each matrix
    norms = np.linalg.norm(matrices, axis=(1, 2))

    # Step 2: Sort the norms
    sorted_indices = np.argsort(norms)

    # Step 3: Compute the bounds for removing outliers
    n_matrices = len(matrices)
    lower_bound = int(n_matrices * outlier_percent)
    upper_bound = int(n_matrices * (1 - outlier_percent))

    # Step 4: Filter matrices
    filtered_indices = sorted_indices[lower_bound:upper_bound]
    filtered_matrices = matrices[filtered_indices]

    # Step 5: Compute the mean and standard deviation after removing outliers
    mean_matrix = np.mean(filtered_matrices, axis=0)
    std_matrix = np.std(filtered_matrices, axis=0)

    return mean_matrix, std_matrix

1


In [8]:
log_folder = "/home/pjlab/.local/share/ov/pkg/isaac-sim-4.0.0/src/frankapy/logs/2024-09-22_17-14-52/log-000001-8404"
log_folder = "/home/pjlab/.local/share/ov/pkg/isaac-sim-4.0.0/src/frankapy/logs/2024-09-22_17-52-45/log-000002-5588"
calibration_eyeinhand_results_dir = "/home/pjlab/Franka_Tool/catkin_ws_2/src/easy_handeye/results/eye_on_hand/9-21--15-38.json"
gs_dir = "/home/pjlab/main/real2sim/gaussian-splatting/data/new/mix2/gs-output/1"
video_output_path = "/home/pjlab/.local/share/ov/pkg/isaac-sim-4.0.0/src/frankapy/tests/test-render/output_video923-6.mp4"
marker_2_base = np.load("/home/pjlab/.local/share/ov/pkg/isaac-sim-4.0.0/src/assets/marker_2_base.npy")

In [ ]:
from scipy.spatial.transform import Rotation as R
from pupil_apriltags import Detector
import os

os.environ["DISPLAY"] = ":1"

# file path
log_folder = Path(log_folder)
hdf5_file = log_folder / "traj.hdf5"
with open(calibration_eyeinhand_results_dir, 'r') as f:
    calibration_results = json.load(f)

hdf5_file = h5py.File(hdf5_file, 'r')
joints = hdf5_file['joints'][:]
timestamps = hdf5_file["timestamp"][:]
fx, fy = hdf5_file["intrinsic_color_1"]["fx"][()], hdf5_file["intrinsic_color_1"]["fy"][()]
image_width, image_height = hdf5_file["intrinsic_color_1"]["width"][()], hdf5_file["intrinsic_color_1"]["height"][()]
print("fx, fy, width, height:", fx, fy, image_width, image_height)
qposes = joints[:, :7]

cam_2_ee = np.eye(4)
quat = np.array([calibration_results["rotation"]["x"], calibration_results["rotation"]["y"], calibration_results["rotation"]["z"], calibration_results["rotation"]["w"]])
cam_2_ee[:3, :3] = R.from_quat(quat).as_matrix()
cam_2_ee[:3, 3] = np.array([calibration_results["translation"]["x"], calibration_results["translation"]["y"], calibration_results["translation"]["z"]])
print(f"cam_EE:\n{cam_2_ee}")

detector =  Detector(families="tagStandard52h13",
                    nthreads=1,
                    quad_decimate=1.0,
                    quad_sigma=0.0,
                    refine_edges=1,
                    decode_sharpening=0.25,
                    debug=0)

franka = rtb.models.Panda()
item_to_show = []
marker_2_bases = []
print("Total num:", len(qposes))
# Define video save path and parameters
frame_rate = 30  # Set frame rate, adjust as needed
fourcc = cv2.VideoWriter_fourcc(*'MP4V')  # Use MP4V encoding

# Initialize VideoWriter object
# Assuming all frames are the same size, use the width and height of the first frame
first_frame = True
video_writer = None
item_to_show = []
gs_2_marker = np.load(Path(gs_dir) / "gs_to_marker.npy")
# Temporary alignment adjustments
marker_2_base[2, 3] -= 0.1
marker_2_base[1, 3] -= 0.036  # Move to the right
marker_2_base[0, 3] += 0.02  # Move forward

for qpos, timestamp in zip(qposes, timestamps):
    ee_2_base = np.array(franka.fkine(qpos))
    cam_2_base = ee_2_base @ cam_2_ee @ rtb.ET.Rz(-np.pi/2).A()
    cam_2_marker = np.linalg.inv(marker_2_base) @ cam_2_base @ rtb.ET.Rx(-np.pi/2).A() @ rtb.ET.Rz(-np.pi/2).A() @ rtb.ET.Rx(-np.pi / 2).A()
    item_to_show.append(show_pose(cam_2_marker, 0.1))
    wrist_camera_pose = np.linalg.inv(gs_2_marker) @ rtb.ET.Rz(np.pi / 2).A() @ cam_2_marker 
    rotation_matrix = wrist_camera_pose[:3, :3] / np.abs((np.linalg.det(wrist_camera_pose[:3, :3]))) ** (1/3)
    # Apply camera pose transformation
    translation = wrist_camera_pose[:3, 3]
    x, y, z, w = R.from_matrix(rotation_matrix).as_quat()
    wrist_camera_pose = [translation, [w, x, y, z]]

    # If it's the first frame, initialize VideoWriter
    if first_frame:
        video_writer = cv2.VideoWriter(
            video_output_path, 
            fourcc, 
            frame_rate, 
            (image_width, image_height)
        )
        first_frame = False

    # Generate image
    image = get_rendered_images(
        wrist_camera_pose,
        image_height,
        image_width,
        fx, 
        fy
    )

    # Ensure the image is in BGR format
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Write frame to video
    video_writer.write(image)
video_writer.release()
frame_base = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])
item_to_show.append(frame_base)
o3d.visualization.draw_geometries(item_to_show)


fx, fy, width, height: 604.0827026367188 602.91064453125 640 480
cam_EE:
[[ 4.68582381e-04 -9.98368542e-01 -5.70967069e-02  8.01850673e-02]
 [ 9.99527826e-01  2.22178170e-03 -3.06461778e-02 -2.74386378e-02]
 [ 3.07230363e-02 -5.70553871e-02  9.97898180e-01 -8.94739605e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Total num: 2330


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
